In [ ]:
#importing
import dbt_functions as dbt
import database_functions as database
import tab_functions as tab
import pandas as pd
from connection_variables import *


In [ ]:
#Authenticating against Snowflake and Tableau.

snow_auth = database.authenticate_snowflake(snow_u, snow_p, snow_account, snow_db)
tab_auth = tab.authenticate_tableau(tab_url,access_token,site_name,token_name)
oracle_auth = database.authenticate_oracle(ora_u=oracle_u, ora_p=oracle_p, ora_ip = oracle_db, ora_service= oracle_service)
dbt_account_id = dbt.get_account_id(dbt_token)
tab_token = tab_auth[0]
site_id = tab_auth[1]

In [ ]:
#Run tab functions to get infos about databases used in Tableau
oracle_infos = tab.get_table_infos(tab_url, oracle_db, tab_token)
snow_infos = tab.get_table_infos(tab_url, snow_db, tab_token)

In [ ]:
#Script for getting a list of descriptions from Oracle and joining them with the matching columns in Tableau
for index, row in oracle_infos.iterrows():
    oracle_comment = database.get_oracle_descriptions(oracle_auth, row["table_name"])
    table_columns = tab.get_list_of_columns(tab_url, row["table_luid"], site_id, tab_token)
    joined = tab.add_comments_to_tab_table(table_columns, oracle_comment)
    tab.update_table_descriptions(tab_url,site_id,joined,row["table_luid"],tab_token)

In [ ]:
#Script for getting a list of descriptions from Snowflake and joining them with the matching columns in Tableau
for index, row in snow_infos.iterrows():
    snow_comment = database.get_snow_descriptions(snow_auth, row["table_name"], row["table_schema"])
    table_columns = tab.get_list_of_columns(tab_url, row["table_luid"], site_id, tab_token)
    joined = tab.add_comments_to_tab_table(table_columns, snow_comment)
    tab.update_table_descriptions(tab_url,site_id,joined,row["table_luid"],tab_token)

In [ ]:
tab.logout(tab_url, tab_auth)